In [0]:
from pyspark.sql.functions import col, to_date, lit
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Load bronze table
bronze = spark.table("mycatalog.default.rent_bronze")

# Filter for non-null rent values
rent_filtered = bronze.filter(col("OriginalValue").isNotNull())

# Filter for 2-bedroom units
rent_2br = rent_filtered.filter(col("Rental Unit Type") == "2 - bedroom")

# Convert PERIOD (year) into a proper date and select relevant columns
rent_silver = (
    rent_2br
    .withColumn("date", to_date(F.concat(col("Period").cast("string"), F.lit("-01-01"))))
    .withColumn("geo", lit("Alberta"))
    .withColumn("rent_value", col("OriginalValue").cast("double"))
    .select("date", "geo", "rent_value")
)

# Average across all cities per year
rent_silver = (
    rent_silver.groupBy("date", "geo")
               .agg(F.avg("rent_value").alias("rent_value"))
               .orderBy("date")
)

# Drop the existing silver table to avoid schema mismatch
spark.sql(
    """
    DROP TABLE IF EXISTS mycatalog.default.rent_silver
    """
)

# Write the province-wide average rent to silver table
rent_silver.write.format("delta").mode("overwrite").saveAsTable("mycatalog.default.rent_silver")

print("✅ rent_silver created (Province-wide average 2-bedroom rent)")
display(spark.table("mycatalog.default.rent_silver").orderBy("date").limit(20))

# Load the latest rent_silver table
rent_silver = spark.table("mycatalog.default.rent_silver")

# Define window for year-over-year comparison
w = Window.partitionBy("geo").orderBy("date")

# Calculate rent_change (YoY % change)
rent_with_change = (
    rent_silver
    .withColumn("prev_rent", F.lag("rent_value").over(w))
    .withColumn(
        "rent_change_percentage",
        F.when(col("prev_rent").isNull(), None)
         .otherwise(100 * (col("rent_value") - col("prev_rent")) / col("prev_rent"))
    )
    .drop("prev_rent")
)
# Add the new column to the Delta table schema
spark.sql(
    """
    ALTER TABLE mycatalog.default.rent_silver
    ADD COLUMNS (rent_change_percentage DOUBLE)
    """
)
# Remove the extra column from the DataFrame to match the table schema
rent_with_change = rent_with_change.drop("rent_change")


# Overwrite the silver table with rent_change included
rent_with_change.write.format("delta").mode("overwrite").saveAsTable("mycatalog.default.rent_silver")

print("🔥 rent_silver updated with rent_change successfully!")
display(rent_with_change.orderBy("date").limit(20))